In [58]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from pprint import pprint
import os
import nltk
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')
nltk.download('omw-1.4')

stemmer = SnowballStemmer(language="english")
lemmatizer = WordNetLemmatizer()

import re
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov|me|edu|ru|ua)"
digits = "([0-9])"
word = "([A-Za-z0-9][A-Za-z0-9]*)"
number = "([1-9][0-9]*)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n>>>>"," ")
    text = text.replace("\n>>>"," ")
    text = text.replace("\n>>"," ")
    text = text.replace("\n>"," ")
    text = text.replace("\n"," ")
    text = re.sub(f"<{word}[.]{word}[.]{word}[.]{word}@{word}[.]{word}[.]{word}>", "<\\1<prd>\\2<prd>\\3<prd>\\4@\\5<prd>\\6<prd>\\7>", text)
    text = re.sub(f"<{word}[.]{word}@{word}[.]{word}[.]{word}>", "<\\1<prd>\\2@\\3<prd>\\4<prd>\\5>", text)
    text = re.sub(f"{word}@{word}[.]{word}[.]{word}", "\\1@\\2<prd>\\3<prd>\\4", text)
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    if "..." in text: text = text.replace("...","<prd><prd><prd>")
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

def split_mail(text):
    head_body = text.split("\n\n")
    return head_body[0], "\n\n".join(head_body[1:]) 

from re import S

files = os.listdir("20news-bydate-train/alt.atheism/")
file = 0
print(files[file])
mail = open("20news-bydate-train/alt.atheism/" + files[file], "r").read()
head, body = split_mail(mail)

def tokenize(sentence):
    #TODO: Можно еще смайлики добавить но мне влом их все перегонять в словарь
    #TODO: Можно еще добавить время 22:11
    sentence = re.sub(f">{word}", "><split>\\1", sentence)

    sentence = re.sub(f"\\({word}", "(<split>\\1", sentence)
    sentence = re.sub(f"{word}\\)", "\\1<split>)", sentence)
    sentence = re.sub(f"I'm", "I<split>am", sentence)

    sentence = re.sub(f"${word}'d", "\\1<split>woud", sentence)
    sentence = re.sub(f"{number},{number}", "\\1<coma>\\2", sentence)

    sentence = re.sub(f"{number},{number},{number}", "\\1<coma>\\2<coma>\\3", sentence)

    sentence = re.sub(f"{number}'{number},{number}", "\\1'\\2<coma>\\3", sentence)

    sentence = re.sub(f"{number}[.]{number}", "\\1<dot>\\2", sentence)
    sentence = re.sub(f"{number}[.]{number}.{number}", "\\1<dot>\\2<dot>\\3", sentence)

    sentence = re.sub(f"{word}[.]{word}[.]{word}@{word}[.]{word}[.]{word}", "<split>\\1<dot>\\2<dot>\\3@\\4<dot>\\5<dot>\\6<split>", sentence)
    sentence = re.sub(f"<{word}[.]{word}@{word}[.]{word}[.]{word}>", "<split>\\1<dot>\\2@\\3<dot>\\4<dot>\\5<split>", sentence)
    sentence = re.sub(f"{word}@{word}[.]{word}[.]{word}", "\\1@\\2<dot>\\3<dot>\\4<split>", sentence)

    sentence = re.sub(f"<{word}@{word}[.]{word}>", "<<split>\\1@\\2<dot>\\3<split>><split>", sentence)
    sentence = re.sub(f"{word}@{word}[.]{word}", "\\1@\\2<dot>\\3<split>", sentence)

    sentence = re.sub(f"<{word}[.]{word}@{word}[.]{word}[.]{word}>", "<split>\\1<dot>\\2@\\3<dot>\\4<dot>\\5<split>", sentence)

    sentence = sentence.replace("...", '<dot><dot><dot>')
    for r in " ":
        sentence = sentence.replace(r, '<split>')
    for r in [".", ",",":",";","?","!", '"', "'", "/", "*", "$"]:
        sentence = re.sub(f"\\{r}{word}", f"{r}<split>\\1", sentence)
        sentence = re.sub(f"{word}\\{r}", f"\\1<split>{r}", sentence)

    sentence = sentence.replace("<dot>", ".")

    sentence = sentence.replace("<coma>", ",")

    tokens = sentence.split('<split>')
    return [(x, stemmer.stem(x), lemmatizer.lemmatize(stemmer.stem(x))) for x in tokens if x]

# maxI = len(split_into_sentences(body)) - 1

# @interact
# def test(i=widgets.IntSlider(min=0,max=maxI,step=1,value=0)):
#     sentence = split_into_sentences(body)[i]
#     pprint(sentence)
#     pprint(tokenize(sentence))

# folders = os.listdir("20news-bydate-test/")


# for folder in folders:
#     files = os.listdir(f"20news-bydate-test/{folder}/")
#     print(folder)
    
#     for file in files:
# # folder = "alt.atheism"
# # file = "49960"

#         if not os.path.exists(f"out/{folder}"):
#             os.makedirs(f"out/{folder}")
#         out = open(f"out/{folder}/{file}.tsv", "w")
#         mail = open(f"20news-bydate-test/{folder}/{file}", "r").read()
#         head, body = split_mail(mail)

#         for sentence in split_into_sentences(head):
#             for token, stem, lem in tokenize(sentence):
#                 out.write(f"{token}\t{stem}\t{lem}\n")
#             out.write("\n")

#         for sentence in split_into_sentences(body):
#             for token, stem, lem in tokenize(sentence):
#                 out.write(f"{token}\t{stem}\t{lem}\n")
#             out.write("\n")

#         out.close()

49960


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ander\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ander\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


# Моя жалкая попытка 2

In [4]:
from datetime import datetime

In [3]:
keyboardArray = [
    ['`','1','2','3','4','5','6','7','8','9','0','-','='],
    ['q','w','e','r','t','y','u','i','o','p','[',']','\\'],
    ['a','s','d','f','g','h','j','k','l',';','\''],
    ['z','x','c','v','b','n','m',',','.','/'],
    ['', '', ' ', ' ', ' ', ' ', ' ', '', '']
    ]

shiftedKeyboardArray = [
    ['~', '!', '@', '#', '$', '%', '^', '&', '*', '(', ')', '+'],
    ['Q', 'W', 'E', 'R', 'T', 'Y', 'U', 'I', 'O', 'P', '{', '}', '|'],
    ['A', 'S', 'D', 'F', 'G', 'H', 'J', 'K', 'L', ':', '"'],
    ['Z', 'X', 'C', 'V', 'B', 'N', 'M', '<', '>', '?'],
    ['', '', ' ', ' ', ' ', ' ', ' ', '', '']
    ]

def arrayForChar(c):
    if (True in [c in r for r in keyboardArray]):
        return keyboardArray
    elif (True in [c in r for r in shiftedKeyboardArray]):
        return shiftedKeyboardArray
    else:
        return None

def getKeyboardXY(c, array):
    if array == None:
        return None
    row = -1
    column = -1
    for r in array:
        if c in r:
            row = array.index(r)
            column = r.index(c)
            return (row, column)
    return None

def get_keyboard_distance(X, Y):
    x = getKeyboardXY(X, arrayForChar(X))
    y = getKeyboardXY(Y, arrayForChar(Y))
    if x == None or y == None:
        return 10
    return ((x[0] - y[0])**2 + (x[1] - y[1])**2)**(0.5)

In [5]:
def get_removal_cost(X):
    return 1

In [6]:
def get_insertion_cost(X):
    return 1

In [7]:
def get_replacement_cost(X, Y):
    return 1 #+ get_keyboard_distance(X, Y)

In [20]:
def distance_vagner_fisher(x, y):
    len_x, len_y = len(x), len(y)
    if len_x > len_y:
        x, y = y, x
        len_x, len_y = len_y, len_x

    current_column = range(len_x + 1)
    for i in range(1, len_y + 1):
        previous_column = current_column
        current_column = [i] + [0] * len_x

        for j in range(1, len_x + 1):
            insert = previous_column[j] + get_insertion_cost(y[i - 1])
            remove = current_column[j - 1] + get_removal_cost(x[j - 1])
            replace = previous_column[j - 1]
        
            if x[j - 1] != y[i - 1]:
                replace += get_replacement_cost(x[j - 1], y[i - 1])
            # if replace > 8:
            #     return None
            current_column[j] = min(insert, remove, replace)

    return current_column[len_x]

a = 'hypothesis'
b = 'reference'

start_time = datetime.now()
dist = distance_vagner_fisher(a, b)
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))
print(dist)

Duration: 0:00:00
9


In [9]:
import os
import csv
import operator

In [10]:
def match(dictionary, word):
    if(len(word) <= 3):
        return 0, word
    dist_min, word_min = 10000, ''
    for item in dictionary:
        dist = distance_vagner_fisher(item, word)
        if dist is not None and dist < dist_min:
            dist_min = dist
            word_min = item

        if dist == 0 : break
    
    if dist_min == 10000:
        return None

    return dist_min, word_min

In [59]:
for folder in os.listdir("test_data/"):
    files = os.listdir(f"test_data/{folder}/")
    for file in files:
        with open(f"out/{file}", "w") as fout:
            with open(f'dicts/{folder}.tsv') as dict_file:
                dictionary = csv.reader(dict_file, delimiter="\t")
                dictionary = [x[0] for x in dictionary if len(x[0]) > 2]
                dictionary.sort(key = lambda x:len(x))
                content = open(f"test_data/{folder}/{file}", "r").read()
                for sentence in split_into_sentences(content):
                    for token, stem, lem in tokenize(sentence):
                        matched =  match(dictionary, token)
                        if matched is not None:
                            print(f"{token}\t{matched[1]}\t{matched[0]}")
                            fout.write(f"{token}\t{matched[1]}\t{matched[0]}\n")
                        else:
                            print(f"{token}\t{token}")
                            fout.write(f"{token}\t{token}\t0\n")
                    fout.write("\n")
            break
    break

From	From	0
:	:	0
decay@cbnewsj.cb.att	decay@cbnewsj.cb.att	0
.	.	0
com	com	0
(	(	0
dea	dea	0
.	.	0
kaflowitz	kaflowitz	0
)	)	0
Subject	Subject	0
:	:	0
Re	Re	0
:	:	0
aboot	about	1
the	the	0
ible	able	1
qyiz	quit	2
answers	answers	0
Organizaion	organization	2
:	:	0
AT&T	ATF	2
Distribution	distribution	1
:	:	0
na	na	0
Linew	new	2
:	:	0
18	18	0
In	In	0
article	article	0
<healta	<healta	0
.	.	0
153.735242337@saturn.wwc.edu	56.734556346@saturn.wwc.edu>	9
>,	>,	0
healta@saturn.wwc.ed	healta@saturn.wwc.edu	1
(	(	0
Tammy	Tammy	0
R	R	0
Heali	Healy	1
)	)	0
writes	writes	0
:	:	0
#12	#12	0
)	)	0
The	The	0
2	2	0
cheribums	serious	4
ae	ae	0
on	on	0
the	the	0
Ark	Ark	0
of	of	0
the	the	0
Cobenant	covenant	2
.	.	0
When	When	0
God	God	0
saod	sod	1
msk	msk	0
no	no	0
raven	haven	1
imae	imam	1
,	,	0
he	he	0
was	was	0
referin	refering	1
to	to	0
idoms	idols	1
,	,	0
which	which	0
were	were	0
created	created	0
to	to	0
be	be	0
worsipped	worshipped	1
.	.	0
The	The	0
Ark	Ark	0
of	of	0
the	the	0
Covenant	covenant	

In [60]:
for file in os.listdir("out/"):
    tsv = open(f"out/{file}").readlines()
    total = len(tsv)
    non_replaced = 0
    for row in tsv:
        t = row.replace("\n", '').split('\t')
        if t[2] == '0':
            non_replaced = non_replaced + 1
    print(non_replaced/total)
    break


IndexError: list index out of range

In [74]:
from pprint import pprint

original = [[]]
replaced = [[]]
before_replaced = [[]]

with open("out/53068") as f_replaced:
    with open("lab1_tokens/53068.tsv") as f_original:
        for a_origibal in f_original.readlines():
            if a_origibal == "\n":
                original.append([])
                continue
            a_origibal = a_origibal.replace("\n", "").split('\t')
            original[-1].append(a_origibal[0].lower())
        
        for b_replaced in f_replaced.readlines():
            if b_replaced == "\n":
                replaced.append([])
                before_replaced.append([])
                continue
            b_replaced = b_replaced.replace("\n", "").split('\t')
            replaced[-1].append(b_replaced[1].lower())
            before_replaced[-1].append(b_replaced[0].lower())
        
        # for word in original:
        #     total = total + 1
        #     if word in replaced:
        #         r = r + 1

# print(total)
# print(r)
# print(r/total)

In [75]:
total = 0
r = 0

for sentence in original:
    for replaced_sentence in replaced:
        if len(sentence) == len(replaced_sentence):
            for word in sentence:
                total = total + 1
                if word in replaced_sentence:
                    r = r + 1
print(total)
print(r)
print(r/total)

150
129
0.86


In [76]:
total = 0
r = 0

for sentence in original:
    for replaced_sentence in before_replaced:
        if len(sentence) == len(replaced_sentence):
            for word in sentence:
                total = total + 1
                if word in replaced_sentence:
                    r = r + 1
print(total)
print(r)
print(r/total)

150
118
0.7866666666666666


In [65]:
original

[['from', ':', 'decay@cbnewsj.cb.att', '.', 'com', '(', 'dean', '.'],
 ['in', 'article', '<healta', '.'],
 ['153.735242337@saturn.wwc.edu',
  '>,',
  'healta@saturn.wwc.edu',
  '(',
  'tammy',
  'r',
  'healy',
  ')',
  'writes',
  ':',
  '#12',
  ')',
  'the',
  '2',
  'cheribums',
  'are',
  'on',
  'the',
  'ark',
  'of',
  'the',
  'covenant',
  '.'],
 ['when',
  'god',
  'said',
  'make',
  'no',
  'graven',
  'image',
  ',',
  'he',
  'was',
  'refering',
  'to',
  'idols',
  ',',
  'which',
  'were',
  'created',
  'to',
  'be',
  'worshipped',
  '.'],
 ['the',
  'ark',
  'of',
  'the',
  'covenant',
  'wasn',
  "'",
  't',
  'wrodhipped',
  'and',
  'only',
  'the',
  'high',
  'priest',
  'could',
  'enter',
  'the',
  'holy',
  'of',
  'holies',
  'where',
  'it',
  'was',
  'kept',
  'once',
  'a',
  'year',
  ',',
  'on',
  'the',
  'day',
  'of',
  'atonement',
  '.'],
 ['i',
  'am',
  'not',
  'familiar',
  'with',
  ',',
  'or',
  'knowledgeable',
  'about',
  'the',
  '

In [66]:
replaced

[['from', ':', 'decay@cbnewsj.cb.att', '.', 'com', '(', 'dea', '.'],
 ['kaflowitz',
  ')',
  'subject',
  ':',
  're',
  ':',
  'about',
  'the',
  'able',
  'quit',
  'answers',
  'organization',
  ':',
  'atf',
  'distribution',
  ':',
  'na',
  'new',
  ':',
  '18',
  'in',
  'article',
  '<healta',
  '.'],
 ['56.734556346@saturn.wwc.edu>',
  '>,',
  'healta@saturn.wwc.edu',
  '(',
  'tammy',
  'r',
  'healy',
  ')',
  'writes',
  ':',
  '#12',
  ')',
  'the',
  '2',
  'serious',
  'ae',
  'on',
  'the',
  'ark',
  'of',
  'the',
  'covenant',
  '.'],
 ['when',
  'god',
  'sod',
  'msk',
  'no',
  'haven',
  'imam',
  ',',
  'he',
  'was',
  'refering',
  'to',
  'idols',
  ',',
  'which',
  'were',
  'created',
  'to',
  'be',
  'worshipped',
  '.'],
 ['the',
  'ark',
  'of',
  'the',
  'covenant',
  'wasn',
  "'",
  't',
  'ripped',
  'znd',
  'only',
  'the',
  'high',
  'priest',
  'could',
  'enter',
  'the',
  'holy',
  'of',
  'lies',
  'where',
  'it',
  'was',
  'kept',
  '